In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=1ff64ab34dfbff06e9c7e7a89432be48cc067607013ed4e45a36a859d87e7742
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import pandas as pd
import sklearn
from sklearn import metrics
import json
import time

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator,BinaryClassificationEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv
model_name = 'distilbert-base-uncased'

model_save_path = '/content/gdrive/MyDrive/685_CP/Models/CL-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
word_embedding_model = models.Transformer(model_name, max_seq_length=64)

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_df = pd.read_excel('../Datasets/Prompting/train-0.3.xlsx')

In [ ]:
from itertools import permutations, combinations
import random
import numpy as np
train_samples = []
hypothesis_list = train_df['Hypothesis'].tolist()
hypothesis_list_train, hypothesis_list_dev, hypothesis_list_test = np.split(hypothesis_list, [int(len(hypothesis_list)*0.95), int(len(hypothesis_list)*0.975)])
#print(hypothesis_list_dev)
hypothesis_list_dev = list(hypothesis_list_dev)
for hypothesis in hypothesis_list_train:
  train_samples.append(InputExample(texts=[hypothesis, hypothesis]))
dev_samples = []
combs = combinations([i for i in range(len(hypothesis_list_dev))],2)
hypothesis_dev_1 = random.sample(hypothesis_list_dev,len(hypothesis_list_dev)//2)
for hypothesis in hypothesis_dev_1:
    score = 1 #Normalize score to range 0 ... 1
    dev_samples.append(InputExample(texts=[hypothesis, hypothesis], label=score))
for comb in combs:
    score = 0
    dev_samples.append(InputExample(texts=[hypothesis_list_dev[comb[0]], hypothesis_list_dev[comb[1]]], label=score))


dev_evaluator = BinaryClassificationEvaluator.from_input_examples(dev_samples, batch_size=8, name='fig-dev')

As loss, we use: MultipleNegativesRankingLoss

Here, texts[0] and texts[1] are considered as positive pair, while all others are negatives in a batch

In [ ]:
# Configure the training
train_batch_size = 32
num_epochs = 8

# Use MultipleNegativesRankingLoss for SimCSE
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)


warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

logging.info("Performance before training")
print(dev_evaluator(model))

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=100,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )


1.0


Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Iteration:   0%|          | 0/223 [00:00<?, ?it/s]

Iteration:   0%|          | 0/223 [00:00<?, ?it/s]

Iteration:   0%|          | 0/223 [00:00<?, ?it/s]

Iteration:   0%|          | 0/223 [00:00<?, ?it/s]

Iteration:   0%|          | 0/223 [00:00<?, ?it/s]

Iteration:   0%|          | 0/223 [00:00<?, ?it/s]

Iteration:   0%|          | 0/223 [00:00<?, ?it/s]

Iteration:   0%|          | 0/223 [00:00<?, ?it/s]

In [ ]:


test_samples = []
hypothesis_list_test = list(hypothesis_list_test)
combs = combinations([i for i in range(len(hypothesis_list_test))],2)
hypothesis_test_1 = random.sample(hypothesis_list_test,len(hypothesis_list_test)//2)
for hypothesis in hypothesis_test_1:
    score = 1 #Normalize score to range 0 ... 1
    test_samples.append(InputExample(texts=[hypothesis, hypothesis], label=score))
for comb in combs:
    score = 0
    test_samples.append(InputExample(texts=[hypothesis_list_test[comb[0]], hypothesis_list_test[comb[1]]], label=score))
model = SentenceTransformer(model_save_path)
test_evaluator = BinaryClassificationEvaluator.from_input_examples(test_samples, batch_size=train_batch_size, name='fig-test')
test_evaluator(model, output_path=model_save_path)

1.0

In [ ]:
model_A = SentenceTransformer('../Models/CL-2023-05-15_08-09-44')

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

In [ ]:
sentence1 = "I was very proud of my sense of judgement when I found out that the person who I admired for their great vision was actually really mean to people"
sentence2 = "I was seriously doubting my sense of judgement when I found out that the person who I admired for their great vision was actually really mean to people"
sentence3 = "It really makes me happy when the person I live with still doesn't want to pay bills on time"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("../Models/CL-2023-05-15_08-09-44")
model = AutoModel.from_pretrained("../Models/CL-2023-05-15_08-09-44")

# Tokenize input texts
texts = [
    sentence1,
    sentence2,
    sentence3
]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True)[1]

In [ ]:
embeddings1 = model_A.encode(sentence1)
embeddings2 =  model_A.encode(sentence2)
embeddings3 = model_A.encode(sentence3)

In [ ]:
print(embeddings[0].shape)

torch.Size([3, 31, 768])


In [ ]:
from scipy.spatial.distance import cosine
print(1-cosine(embeddings1,embeddings2))
print(1-cosine(embeddings1,embeddings3))
print(1-cosine(embeddings2,embeddings3))

0.8484863638877869
-0.01885334402322769
-0.00479287002235651


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
texts = [
    sentence1,
    sentence2,
    sentence3
]
sentence_embeddings = model.encode(texts)
print(util.cos_sim(sentence_embeddings[0],sentence_embeddings[1]))
print(util.cos_sim(sentence_embeddings[0],sentence_embeddings[2]))
print(util.cos_sim(sentence_embeddings[1],sentence_embeddings[2]))

tensor([[0.8969]])
tensor([[0.1990]])
tensor([[0.1280]])
